# Bayesian Networks

In [23]:
import math 
from pomegranate import *
import json
from copy import copy

#### Function addNewTable creates all the Conditional Probability Tables of the Bayesian Network

In [24]:
def addNewTable(node, cond_dis_list):
    
    ##List that contain the distributions and conditionals of the Conditional Probability Table
    conditional_list = []
    distribution_list = []
    
    ##Reads every k in the node
    for k in node.keys():
        
        ##List that contain the probability table of the Conditional Probability Table
        probability_list = []
        complementaryProb_list = []
        
        ##Info has the informations of the name and parents of the state 
        info = k.split("|")
        
        ##Parents has the list of parents of the state 
        parents = info[1].split(",")
        
        ##For every parent of the k it will push it to the probability list
        for i in range(len(parents)):
            probability_list.append(parents[i])
         
        ##Adds parent to probability list
        probability_list.append(info[0])
        
        ##It pushes the numerical value of the probability 
        probability_list.append(node.get(k))
        
        #print("OG table", probability_list)
        
        ##Adds that probability to the table
        #print("ADDING to conditional list:", probability_list)
        conditional_list.append(probability_list)
        
        #Adds complementary row to actual probability_list
        complementaryProb_list = copy(probability_list)
        complementaryProb_list[len(complementaryProb_list)-2] = "-" + complementaryProb_list[len(complementaryProb_list)-2]
        complementaryProb_list[len(complementaryProb_list)-1] = 1 - complementaryProb_list[len(complementaryProb_list)-1]
        #print("NEW table", complementaryProb_list)
        #print("ADDING to conditional list:", probability_list)
        conditional_list.append(complementaryProb_list)
        #print("\nMYPROBABILITYLIST:\n", conditional_list)
   
    ##Reads every k in the node
    for k in node.keys():
        
        ##Info has the informations of the name and parents of the state 
        info = k.split("|")
        
        ##Parents has the list of parents of the state 
        parents = info[1].split(",")
        
        ##For every parent of the k it will push it to the distribution list
        for i in range(len(parents)):
            
            ##For every distribution that equals to the parents  will push it to the distribution list
            for d in range(len(cond_dis_list)):
                if parents[i] == cond_dis_list[d][0] or parents[i] == "-"+cond_dis_list[d][0][0]:
                    distribution_list.append(cond_dis_list[d][1])
                    #print("ELEMENT:", cond_dis_list[d][1], "appended!")
                    break 
            
            ##For every distribution that equals to the parents  will push it to the distribution list
        
        break
    
    #print(distribution_list)
    #print(conditional_list)
    
    ##Creates a Conditional Probability Table with the conditional_list and distribution_list generated in the function
    cond = ConditionalProbabilityTable(conditional_list,distribution_list)
    #print("salu2\n", cond)
    
    ##Returns the Conditional Probability Table cond
    return cond

#### Function addEdgesOfStates adds all the Edges between states of the Bayesian Network 

In [25]:
#tuples of length 2 = DISCRETE DISTRIBUTION (NO PARENTS) -> (name, distribution)
#tuples of length 3 = CONDITIONAL PROBABILITY TABLE (WITH PARENTS) -> (name, cond, parents)

def addEdgesOfStates(network, cond_dist_list, state_list):
    cond_dist_list_INDEX = 0;
    for condDist in cond_dist_list:
        #A tuple of length 3 indicates that the distribution consists of a conditional probability table,
        #and thus has a parent which creates an edge with someone else.
        if(len(condDist) == 3):
            #The "son" state will always correspond with the current index of the for loop
            son = state_list[cond_dist_list_INDEX]
            #For each parent found to be associated with X son
            for parentTag in condDist[2]:
                #Searches for STATE associated with parentTag
                for parent in state_list:
                    if(parent.name == parentTag):
                        #Adds edge to the network
                        #print("Parent to be added:\n", parent, "\nSon to be added:\n", son)
                        network.add_edge(parent, son)
        cond_dist_list_INDEX += 1
    
    return network

#### Function createJsonFile creates a JSON file with the Bayesian Network

In [4]:
def createJsonFile():
    

SyntaxError: unexpected EOF while parsing (<ipython-input-4-abdf74602805>, line 2)

In [26]:
with open("Noisy_OR.json", "r") as f:
    bayer_data = json.load(f)

##Lists that contain the distributions, conditionals and states of the Bayesian Network
cond_dist_list = []
state_list = []
conditional_list = []

counter = 0


##Reads every line of the JSON file 
for node in bayer_data:
    
    ##Reads every key value from the node that is reading at the moment
    for k in node.keys():
        
        ##If the k does not have a character "|" it means is a Discrete Distribution
        if k.find("|") == -1:
            name = k[0:len(k):1]
            ##Creates a Discrete Distribution with the k value as the name and the information of the key as the rest and adds it to the dictionary
            negativeKey = "-" + name + ""
            negativeVal = 1 - node[k]
            distribution = DiscreteDistribution({k : node.get(k), negativeKey: negativeVal})
            tuple_dis =(name, distribution)
            cond_dist_list.append(tuple_dis)
            ##Creates a new state with the distribution and the name k
            state_list.append(State( distribution, name = k))
            
        ##If the k has a "|" it means is a Conditional Probability Table    
        elif k.find("|") == 1: 
            
            info = k.split("|")
            name = info[0]
            parents = info[1].split(",")
            ##Calls the function addNewTable that returns a Conditional Probability Table and adds it to the dictionary
            cond = addNewTable(node, cond_dist_list)
            tuple_cond= (name,cond,parents)
            cond_dist_list.append(tuple_cond)
           
            ##Creates a new state with the conditional and the name k
            state_list.append(State(cond, name = k.split("|")[0]))
            #print(cond)
            
            break

#Creates network and adds all States in state_list to it
network = BayesianNetwork("Name of Bayesian Network")
for stateElement in state_list:
    network.add_states(stateElement)

#Adds all edges to the network
network = addEdgesOfStates(network, cond_dist_list, state_list);

network.bake()

#TO-DO agregar beliefs predict proba statement {disque Fax}
#H true, I false, query B
beliefs = network.predict_proba({'D' : 'D', 'H' : '-H'})
print("n".join( "{}t{}".format( state.name, str(belief) ) for state, belief in zip( network.states, beliefs )))
#TO-DO agregar PRINT statement  {disque Fax???}
#TO-DO checar contra la rúbrica

#Este es la entrega final.

#DEBUG AREA
#print(network)

At{
    "class" :"Distribution",
    "dtype" :"str",
    "name" :"DiscreteDistribution",
    "parameters" :[
        {
            "A" :0.44998917469890237,
            "-A" :0.5500108253010975
        }
    ],
    "frozen" :false
}nBt{
    "class" :"Distribution",
    "dtype" :"str",
    "name" :"DiscreteDistribution",
    "parameters" :[
        {
            "B" :0.04999378793346677,
            "-B" :0.9500062120665332
        }
    ],
    "frozen" :false
}nCt{
    "class" :"Distribution",
    "dtype" :"str",
    "name" :"DiscreteDistribution",
    "parameters" :[
        {
            "C" :0.009999162444558372,
            "-C" :0.9900008375554417
        }
    ],
    "frozen" :false
}nDtDnEt{
    "class" :"Distribution",
    "dtype" :"str",
    "name" :"DiscreteDistribution",
    "parameters" :[
        {
            "-E" :0.9490050938945572,
            "E" :0.05099490610544277
        }
    ],
    "frozen" :false
}nFt{
    "class" :"Distribution",
    "dtype" :"str",
    "name"

In [ ]:
hola = [1,2,3]
print(len(hola))